In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com/ubuntu focal-securi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [5]:
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [29]:
vine_table_df.count()

2302401

In [8]:
from pyspark.sql.functions import col
filtered_vine = vine_table_df.filter(col("total_votes") >= 20)
filtered_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [11]:
helpful_vine = filtered_vine.filter((filtered_vine.helpful_votes/filtered_vine.total_votes)>=0.5)
helpful_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [14]:
#Retrieve all the rows where a review was written as part of the Vine program (paid)
paid_df = helpful_vine.filter(helpful_vine.vine == "Y")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|          2|           15|         21|   Y|                N|
|R1XWEI58A7VKLB|          5|          123|        128|   Y|     

In [16]:
#retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_df = helpful_vine.filter(helpful_vine.vine == "N")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [18]:
#Total paid reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

107

In [19]:
#Total unpaid reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

39869

In [23]:
#Total 5-star paid reviews
total_5_star_paid_reviews = paid_df.filter(paid_df.star_rating == 5).count()
total_5_star_paid_reviews

56

In [24]:
#Total 5-star unpaid reviews
total_5_star_unpaid_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
total_5_star_unpaid_reviews

21005

In [26]:
#percent 5-star paid reviews
percent_5_star_paid_reviews = (total_5_star_paid_reviews/total_paid_reviews)*100
percent_5_star_paid_reviews

52.336448598130836

In [27]:
#percent 5-star unpaid reviews
percent_5_star_unpaid_reviews = (total_5_star_unpaid_reviews/total_unpaid_reviews)*100
percent_5_star_unpaid_reviews

52.68504351751988